# Attention with trainable weights

In [35]:
import torch

In [36]:
inputs = torch.nn.Embedding( 4, 8 )

In [37]:
inputs = inputs.weight.data
inputs

tensor([[-1.0549, -1.2410, -1.2727, -0.9202,  0.5775,  0.7022,  0.8694, -0.0043],
        [-0.3531, -0.6784,  1.2160, -0.6186, -0.6871, -0.4007,  0.3720, -0.2599],
        [-0.7226, -0.1292, -0.7689, -0.9081, -0.7946, -1.1334,  1.0931,  0.2480],
        [-1.0707, -0.0070,  1.8550,  1.9543, -1.6581, -0.5846,  0.4483,  0.0652]])

In [38]:
# set dimension
d_in = 8
d_out = 6

# create weight matrices
w_query = torch.nn.Parameter( torch.randn( d_in, d_out ), requires_grad=False )
w_key = torch.nn.Parameter( torch.randn( d_in, d_out ), requires_grad=False )
w_value = torch.nn.Parameter( torch.randn( d_in, d_out ), requires_grad=False )


In [39]:
# choose and input vector and tranfrom it into our query vector using w_query
query = inputs[2] @ w_query
query

tensor([ 1.4573, -2.1323, -2.3605, -2.6233,  2.6573,  0.4844])

In [40]:
# calculate attention scores using the keys generated by w_key
keys = inputs @ w_key
values = inputs @ w_value
print("Values: ", values, "\nKeys: ", keys)

Values:  tensor([[-0.8270, -0.3307,  0.2861,  1.5011, -0.8435,  3.2534],
        [-4.1158, -5.7532,  1.7549,  0.0750, -1.8873, -0.0725],
        [-2.6695, -5.6879,  0.3235, -2.8856, -4.8177,  0.2529],
        [-1.7539, -3.7876,  1.6834,  2.1608,  0.4728, -7.1969]]) 
Keys:  tensor([[ 0.1711, -3.8989, -1.2842, -1.2390,  4.1545, -3.1930],
        [-2.5356, -1.7860, -1.4798,  0.7440,  2.1231,  1.0812],
        [-2.3679, -2.3318, -2.0670, -0.8711,  0.9281,  1.1790],
        [-5.7742, -1.8014, -1.3338,  1.0721, -0.3762,  0.9523]])


In [52]:
attention_scores = query @ keys.T
attention_scores

tensor([24.3375,  7.8197, 11.7230, -4.7759])

In [53]:
# attention_weights = attention_scores.softmax(dim = -1)

attention_weights = torch.softmax(attention_scores, dim=-1 )
attention_weights

tensor([1.0000e+00, 6.7056e-08, 3.3236e-06, 2.2709e-13])

In [54]:
attention_weights.sum()

tensor(1.0000)

In [55]:
context_vector = attention_weights @ values
context_vector

tensor([-0.8270, -0.3307,  0.2861,  1.5011, -0.8435,  3.2534])